In [1]:
print("All Ok")

All Ok


In [2]:
from dotenv import load_dotenv
import os
load_dotenv()


True

In [6]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [7]:
if not OPENAI_API_KEY:
    raise ValueError("OPENAI_API_KEY is not set in the environment variables.")
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [3]:
from langchain_openai import ChatOpenAI

In [4]:
chat_llm = ChatOpenAI(model="gpt-4o-mini")

In [5]:
chat_llm.invoke("Hello, how are you?").content

"Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?"

In [8]:
from typing_extensions import TypedDict, Annotated
import operator

In [10]:
from langchain_core.messages import AnyMessage, HumanMessage, AIMessage

In [11]:
class GraphState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

In [ ]:
["hi"]

In [12]:
def llm_call(state: GraphState) -> dict:
    response = chat_llm.invoke(state["messages"])
    return {"messages":  [response]}

In [13]:
def token_counter(state: GraphState) -> dict:
    last_msg = state["messages"][-1]
    text = last_msg.content
    token_number = len(text.split())
    summary = f"Total token number in the generated answer (word count) is {token_number}"
    return {"messages": [AIMessage(content=summary)]}